# Obteniendo datos de twitter de candidatos

Recientemente [SocialTic](https://socialtic.org/) liberó [API Candidaturas MX](https://socialtic.org/blog/api-candidaturas-mx/), una API para obtener información de ~2,500 candidatos que participarán en las elecciones mexicanas de 2021. La API cuenta con información de candidatos a diputaciones federals, gubernaturas y presidencias municipales. En este documento jugamos un poco con la API y a manera de tutorial mostramos como se puede consumir desde Python y cruzar sus datos con datos obtenidos a partir de Twitter.

Para seguir este tutorial es necesario que vayas siguiendo el código en un Notebook de jupyter (este tutorial no hablará de como instalar jupyter ni de como echarlo a andar, pero puedes seguir [este tutorial](https://github.com/irvingfisica/covid-bokeh) y aprender un poco de análisis de datos y graficación en python). Además necesitaras obtener un token de consulta para la API de SocialTic y un token de consulta para la API de twitter. Además haremos manejo de JSONs y crearemos algunas funciones en python, por lo tanto el tutorial asume que sabes un poco de Python. Si hay alguna instrucción que no entiendes no dudes en preguntarme por MD en mi cuenta de Twitter: [@moaimx](https://twitter.com/moaimx).

## Obteniendo un token para la API Candidaturas MX

El primer paso para poder jugar con los datos es obtener un token para tener acceso a la API Candidaturas MX. Para ello tenemos que ingresar en la página de la [API](https://www.apielectoral.mx/) y seguir las instrucciones para obtener un token. Tendras que crear un usuario en la plataforma y acceder con estas credenciales. Una vez que ingresamos el sistema nos dará un token válido por 4 horas. ESta cadena de texto será la que ocuparemos para hacer peticiones a la API. 

![token](./s1.png)

Para hacer peticiones a la API utilizaremos la librería 'requests' y para poder escribir nuestros datos y leerlos en un futuro usaremos la librería 'json':

In [1]:
import requests 
import json 

Necesitamos guardar nuestro id_token en una variable. Ya que esto es un tutorial lo haremos directamente en el notebook, sin embargo esta no es la mejor práctica en cuanto a seguridad pues si alguien más tiene acceso a tu notebook podría usar tu token sin tu consentimiento, existen otras maneras de manejar tus tokens, por ejemplo guardarlos como 'enviroment variables' en tu sistema y consumirlas usando la librería 'os'.

In [ ]:
token_id = "COPIA AQUÍ TU TOKEN_ID"

Ya que tenemos un token_id ilsto necesitamos pensar que tipo de peticiones haremos a la API. La API tiene un montòn de endpoints con informaciòn ùtil. Usaremos el endpoint 'export' que de acuerdo a la pàgina de instrucciones de la API nos da acceso a *Toda la informaciòn*. Si copiamos el enlace al que hace referencia la pàgina de instrucciones de la API podemos obtener la direcciòn del endpoint:

In [8]:
endpoint = 'https://e7f1hlosbh.execute-api.us-east-2.amazonaws.com/staging/export'

Obtendremos los datos del endpoint a traves de una peticiòn de tipo *GET* para lo cual necesitamos construir un header con nuestro *token_id* en el campo 'Authorization'

In [10]:
headers = {'Authorization': token_id}

Con este header ya podemos hacer la peticiòn:

In [11]:
r = requests.get(endpoint, headers=headers)

Para checar si nuestra peticiòn se completò de forma correcta podemos checar el còdigo de status que fue respondido, si el còdigo es '200' la peticiòn se ha resuelto de forma satisfactoria:

In [13]:
print(r.status_code)

200


Asociemos la respuesta de la peticiòn a una variable para que podamos jugar con ellos...

In [14]:
datos = r.json()

Tambièn podemos guardar los datos en un archivo de tipo JSON para que podamos usar estos datos despuès sin tener que hacer una nueva peticiòn:

In [15]:
with open('./datos_api.json', 'w') as outfile:
    json.dump(datos, outfile, indent=4, sort_keys=True)

Una vez que tenemos nuestros datos podemos entonces empezar a explorarlos y a jugar con ellos. La respuesta de la peticiòn es un JSON que la librerìa requests se encarga de convertir en un dict de python. Primero checaremos que *llaves* tienen nuestros datos:

In [17]:
datos.keys()

dict_keys(['areas', 'chambers', 'coalitions', 'contests', 'memberships', 'other-names', 'parties', 'person-professions', 'persons', 'professions', 'roles', 'urls'])

Podemos observar que estas *llaves* se corresponden con los diferentes endpoints accesibles en la página de instrucciones de la API:

- 'persons': Personas
- 'memberships': Adscripciones políticas
- 'contests': Contiendas políticas
- 'parties': Partidos políticos
- 'coalitions': Coaliciones
- 'areas': Áreas
- 'chambers': Cámaras
- 'roles': Roles
- 'other-names': Otros nombres por persona 
- 'person-professions': Profesiones por persona 
- 'professions': Profesiones 
- 'urls': URLs

Con lo cual efectivamente el endpoint que seleccionamos tiene acceso a todos los datos de la API

Exploremos el campo 'persons'. Podemos ver que contiene una lista y que además tiene 2566 registros:

In [20]:
type(datos['persons'])

list

In [21]:
len(datos['persons'])

2566

Si exploramos el primer registro podemos ver que cada registro contiene la información de un candidato:

In [40]:
datos['persons'][2]

{'contest_id': 301,
 'date_birth': '1967-09-06',
 'dead_or_alive': True,
 'fb_urls': [{'note': 'campaign',
   'url': 'https://www.facebook.com/LupitaJonesOficial'}],
 'first_name': {'en_US': 'María Guadalupe', 'es_MX': 'María Guadalupe'},
 'full_name': {'en_US': 'María Guadalupe Jones Garay',
  'es_MX': 'María Guadalupe Jones Garay'},
 'gender': 'F',
 'id': 3,
 'ig_urls': [{'note': 'campaign',
   'url': 'https://www.instagram.com/lupjones'}],
 'last_degree_of_studies': 'MASTER DEGREE',
 'last_name': {'en_US': 'Jones Garay', 'es_MX': 'Jones Garay'},
 'other_names': {'ballot_name': [],
  'nickname': [{'en_US': 'Lupita Jones'}, {'es_MX': 'Lupita Jones'}],
  'preferred_name': []},
 'photo_urls': ['https://scontent.fmex10-4.fna.fbcdn.net/v/t1.6435-9/182714447_312887933537451_3350126348236012989_n.jpg?_nc_cat=1&ccb=1-3&_nc_sid=09cbfe&_nc_eui2=AeHwyYtq5t7Ik03MSCDhzG-EYKvTvUNUrKhgq9O9Q1SsqK58DA_Ktm0AGdzLKVwmlma8axpvHBarSbxaS6hUDsm7&_nc_ohc=SPD2HH4SLI0AX8pNugX&_nc_ht=scontent.fmex10-4.fna&oh=0d

Hay muchos datos interesantes, por ejemplo el último grado de estudios o las profesiones con las que cuenta cada candidato. Para este tutorial nos concentraremos en el campo 'social_network_accounts' que como podemos observar contiene una lista cuyos elementos son otro dict. Cada elemento está asociado a una cuenta de redes sociales perteneciente al candidato. En particular nos interesa la cuenta de twitter. En particular este campo tiene la url asociada a la cuenta de la persona de la cual podemos obtener el nombre de usuario que será nuestro acceso a los datos de la cuenta. 

Para obtener los datos de twitter asociados a la cuenta usaremos la API de twitter. Para usar esta API es necesario tener una cuenta en twitter y darnos de alta para el programa de developers. Esto se puede hacer en [este enlace](https://developer.twitter.com/en/apply-for-access). El proceso para darse de alta como developer dura aproximadamente una semana, en el te preguntarán para que quieres acceso a la API, para que la utilizarás. Es importante que seas explícito en el tipo de uso que le darás a la API y que estes completamente seguro que el uso que le darás no viola los términos y condiciones de la plataforma. En el proceso de darse de alta puede ser que alguien de Twitter te escriba un mail preguntando algunas cosas puntuales del uso que pretendes darle a tu aplicación. 

En mi caso el proceso fue rápido pero involucró rebotar un par de correos con ellos. Quizá te desanimes al ver que este proceso es necesario, sin embargo una vez que tu aplicación es aprobada puedes jugar con la API de twitter y con sus datos. Acá dejo un [tutorial](https://dev.to/sumedhpatkar/beginners-guide-how-to-apply-for-a-twitter-developer-account-1kh7) donde explican algunos detalles del proceso.

UNa vez que tengas acceso a la plataforma de developer de twitter tendrás que crear un 'project' y crear una 'app' en el. También puedes crear 'apps' fuera de un proyecto, sin embargo por ahora estas apps no tienen acceso a la versión 2.0 de la API de twitter. Acá encuentras toda la [información para crear un proyecto y una app en el](https://developer.twitter.com/en/docs/projects/overview).

Una vez que tienes creada tu app tienes que dirigirte a la sección de 'keys and tokens' para obtener un 'bearer token'. Lo copiaremos y guardaremos en una variable para poder usarlo. Como mencioné anteriormente, esta no es la mejor práctica en cuanto a seguridad y no deberías de dejar tu token en el notebook, especialmente si piensas compartirlo de alguna manera.

In [ ]:
bearer_token = 'COPIA AQUÏ EL BEARER_TOKEN DE TU APP'

De manera similar a como lo hicimos con la API de datos electorales haremos requests a la API de twitter para obtener información de las cuentas que nos interesan. Para esto necesitamos crear el header que acompañará nuestras peticiones. En el agregaremos nuestro token para poder identificarnos con la API.

In [26]:
twitter_header = {"Authorization": "Bearer {}".format(bearer_token)}

Para poder obtener información acerca de un usuario de twitter con su API necesitamos usar el endpoint ['User lookup'](https://developer.twitter.com/en/docs/twitter-api/users/lookup/introduction). Este endpoint nos permite obtener información de cuentas de twitter. En particular usaremos el endpoint específico para ibtener info de usuarios de tiwtter identificandolos a traves de su nombre de usuario. Puedes encontrar la documentación [aquí](https://developer.twitter.com/en/docs/twitter-api/users/lookup/api-reference/get-users-by).

El endpoint soporta peticiones de hasta 100 usuarios en una misma petición, sin embargo tiene un límite de peticiones de hasta 300 por cada 15 minutos o 900 si nos identificamos a traves de un token de usuario y no de un token de app. La url del endpoint es 'https://api.twitter.com/2/users/by' y debe de llevar como parámetro los nombres de usuarios de los cuales queremos obtener la información separados por comas. Como primera prueba usaremos el nombre de usuario de una sola persona en nuestra lista de personas: 'LupJonesof'



In [41]:
user_url = 'https://api.twitter.com/2/users/by'
user_params = {'usernames':'LupJonesof'}

Una vez que hemos construido el header y los parametros a utilizar podemos realizar nuestra petición a traves de la librería 'requests'. La petición será una petición de tipo GET.

In [42]:
rtw = requests.request("GET", user_url, headers=twitter_header, params=user_params)

In [43]:
print(rtw.status_code)

200


Si exploramos que datos fueron regresados en la petición podemos ver que solamente obtuvimos el nombre del usuario, su id y el suername que utilizamos para realizar la petición. 

In [44]:
rtw.json()

{'data': [{'id': '111468543',
   'name': 'Lupita Jones',
   'username': 'LupJonesof'}]}

Sin embargo el endpoint es capaz de proporcionarnos muchos más datos asociados a una cuenta de twitter, solamente hay que pedirselos. En específico se pueden pedir dos tipos de datos, datos asociados a la cuenta y expansiones, en este caso la única expansión disponible es información del tweet 'pinneado' en el perfil si es que existe. Para este tweet se puede pedir información.

Con respecto a la información del usuario podemos pedir:
- la fecha de creación con el parámetro 'created-at'
- la descripción de la cuenta con 'description'
- entidades encontradas en el texto por twitter con siginificados especiales con el parámetro 'entities'
- el id de la cuenta con el parámetro 'id' (este es muy importante si pensamos obtener posteriormente más datos del usuario como por ejemplo tweets de su timeline)
- el lugar asociado a la cuenta con el parámetro 'location'
- el nombre del usuario con 'name'
- el id del tweet pinneado con 'pinned-tweet-id'
- una URL con acceso a la imágen asociada al perfil con 'profile-image-url'
- si el usuario ha protegido sus tweets o no con 'protected'
- las métricas públicas de la cuenta como followers, following y tweets emitidos con 'public_metrics'
- la URL que el usuario ha asociado a su cuenta con 'url'
- el nombre del usuario con 'username'
- si la cuenta está verificada o no con 'verified'

Con respecto a la información del tweet pinneado podemos pedir:
- los objetos adjuntos al tweet, por ejemplo encuestas o imágenes con 'attachments'
- el id del autor con 'author_id'
- anotaciones de contexto encontradas por twitter en el texto del tweet, por ejemplo lugares, personas o marcas con 'context_annotations'
- el id de la conversación a la que pertenece el tweet con 'conversation_id'
- la fecha de creación del tweet con 'created_at'
- entidades encontradas en el texto por twitter con siginificados especiales con el parámetro 'entities'
- la información geográfica del tweet con 'geo'
- el id del tweet con 'id'
- el usuario al cual se le respondió en el tweet si es que este es una respuesta con 'in_reply_to_user_id'
- el idioma del tweet con 'lang'
- algunas métricas públicas del tweet con 'public_metrics' (también es posible obtener métricas no públicas si eres dueño del tweet)
- si el tweet está catalogado como portador potencial de contenido sensible con 'possibly_sensitive'
- los tweets a los que hace referencia con 'referenced_tweets'
- las propiedades de respuesta del tweet con 'replay_settings'
- la aplicación con la cual se emitió el tweet con 'source'
- el texto del tweet con 'text'

Como podemos ver la API da acceso a un montón de información, en este ejmplo pediremos toda la información possible. Para eso tenemos que modificar nuestra variable de parámetros. Para pedir el tweet pinneado usamos el campo 'expansions' con el valor 'pinned_tweet_id', para pedir datos de usuario usamos el campo 'user.fields' con un valor igual a una cadena que contiene todos los parámetros que nos gustaría obtener separados por comas y sin espacios entre ellos. De manera similar para pedir datos del tweet pinneado usamos el campo 'tweet.fields' con una cadena con los parámetros requeridos como valor:

In [66]:
usernames = ['LupJonesof']
expansions = ['pinned_tweet_id']
tweet_fields = ['attachments',
                'author_id',
                'context_annotations',
                'conversation_id',
                'created_at',
                'entities',
                'geo',
                'id',
                'in_reply_to_user_id',
                'lang',
                'public_metrics',
                'possibly_sensitive',
                'referenced_tweets',
                'reply_settings',
                'source,text']
user_fields = ['created_at',
               'description',
               'entities',
               'id',
               'location',
               'name',
               'pinned_tweet_id',
               'profile_image_url',
               'protected',
               'public_metrics',
               'url',
               'username',
               'verified']

In [67]:
usernames_param = ','.join(usernames)
expansions_param = ','.join(expansions)
tweet_params = ','.join(tweet_fields)
user_params = ','.join(user_fields)

In [68]:
parametros = {'usernames':usernames_param,
        "expansions": expansions_param,
        "tweet.fields": tweet_params,
        "user.fields": user_params}

y volvemos a hacer nuestra petición con estos nuevos parámetros:

In [69]:
rtw = requests.request("GET", user_url, headers=twitter_header, params=parametros)

In [70]:
print(rtw.status_code)

200


Al explorar la respuesta de nuestra petición podemos observar que esta tiene dos campos, el campo 'data' y el campo 'includes'. Los datos de los usuarios estarán en 'data' y los datos de los tweets asociados estarán en 'includes' en el subcampo 'tweets':

In [71]:
rtw.json().keys()

dict_keys(['data', 'includes'])

In [72]:
datos_usuario = rtw.json()['data'] 
datos_usuario

[{'profile_image_url': 'https://pbs.twimg.com/profile_images/1378604052532101120/nKkwP19z_normal.jpg',
  'name': 'Lupita Jones',
  'description': 'Orgullosa cachanilla, candidata  a la gubernatura de Baja California por la coalición “Va por Baja California”.',
  'url': '',
  'pinned_tweet_id': '1384017997635809281',
  'created_at': '2010-02-05T01:27:17.000Z',
  'protected': False,
  'verified': True,
  'id': '111468543',
  'public_metrics': {'followers_count': 160976,
   'following_count': 292,
   'tweet_count': 21818,
   'listed_count': 337},
  'username': 'LupJonesof'}]

In [73]:
datos_tweet = rtw.json()['includes']['tweets']
datos_tweet

[{'created_at': '2021-04-19T05:36:19.000Z',
  'author_id': '111468543',
  'entities': {'urls': [{'start': 280,
     'end': 303,
     'url': 'https://t.co/RcRYs8sXqx',
     'expanded_url': 'https://twitter.com/LupJonesof/status/1384017997635809281/video/1',
     'display_url': 'pic.twitter.com/RcRYs8sXqx'}],
   'annotations': [{'start': 165,
     'end': 172,
     'probability': 0.9764,
     'type': 'Place',
     'normalized_text': 'Mexicali'}],
   'hashtags': [{'start': 240, 'end': 253, 'tag': 'EstadoModelo'},
    {'start': 255, 'end': 264, 'tag': 'BCVota21'},
    {'start': 265, 'end': 279, 'tag': 'DebatesBC2021'}]},
  'lang': 'es',
  'text': 'El próximo 6 de junio tienes 3 opciones: votar por un candidato que no sabemos si tiene buenas o malas intenciones, elegir a una candidata que dejó en el abandono a Mexicali; o votar por mí, una ciudadana que, de tu mano, va a construir un #EstadoModelo. #BCVota21 #DebatesBC2021 https://t.co/RcRYs8sXqx',
  'id': '1384017997635809281',
  'possibly_

Para poder obtener los datos de todas las candidatas y candidatos necesitamos hacer peticiones para todos ellos. Primero necesitamos saber cuales de ellos tienen una cuenta de twitter asociada en la información que obtuvimos de la API electoral. Analicemos de nuevo la info que tenemos de cada persona

In [74]:
datos['persons'][2]

{'contest_id': 301,
 'date_birth': '1967-09-06',
 'dead_or_alive': True,
 'fb_urls': [{'note': 'campaign',
   'url': 'https://www.facebook.com/LupitaJonesOficial'}],
 'first_name': {'en_US': 'María Guadalupe', 'es_MX': 'María Guadalupe'},
 'full_name': {'en_US': 'María Guadalupe Jones Garay',
  'es_MX': 'María Guadalupe Jones Garay'},
 'gender': 'F',
 'id': 3,
 'ig_urls': [{'note': 'campaign',
   'url': 'https://www.instagram.com/lupjones'}],
 'last_degree_of_studies': 'MASTER DEGREE',
 'last_name': {'en_US': 'Jones Garay', 'es_MX': 'Jones Garay'},
 'other_names': {'ballot_name': [],
  'nickname': [{'en_US': 'Lupita Jones'}, {'es_MX': 'Lupita Jones'}],
  'preferred_name': []},
 'photo_urls': ['https://scontent.fmex10-4.fna.fbcdn.net/v/t1.6435-9/182714447_312887933537451_3350126348236012989_n.jpg?_nc_cat=1&ccb=1-3&_nc_sid=09cbfe&_nc_eui2=AeHwyYtq5t7Ik03MSCDhzG-EYKvTvUNUrKhgq9O9Q1SsqK58DA_Ktm0AGdzLKVwmlma8axpvHBarSbxaS6hUDsm7&_nc_ohc=SPD2HH4SLI0AX8pNugX&_nc_ht=scontent.fmex10-4.fna&oh=0d

Como mencionamos anteriormente el campo que nos interesa es 'social_network_accounts' en específico cuando este campo contiene una entrada correspondiente a una cuenta de Twitter. Al realizar este tutorial noté que algunas de las entradas que tienen 'type' igual a 'Twitter' tienen URLs correspondientes a otras redes sociales (esto quizá ya está corregido en la actualización de datos de la API electoral), debido a esto lo mejor es detectar la información de una cuenta de twitter a traves de la URL y no del tipo de cuenta. Hagamos una función que tenga como entrada la información de una persona y nos regrese el nombre de usuario de twitter de la persona en caso de existir. En caso de no existir nuestra función deberá regresar 'None'. Primero checaremos si el campo 'social_network_accounts' existe y luego filtraremos su contenido para obtener una cuenta de twitter en caso de que exista. Despues usaremos la URL asociada para obtener el nombre de usuario. Escribamos la función y luego la explicamos paso a paso:

In [75]:
def usuario_twitter(datos_persona):
    if datos_persona.get('social_network_accounts'):
        cuentas_redes = datos_persona['social_network_accounts']
        cuentas_de_twitter = list(filter(lambda x: x.get('value').find('twitter.com') != -1,cuentas_redes))
        if len(cuentas_de_twitter) >= 1:
            url = cuentas_de_twitter[0]['value']
            extra_pos = url.find('?')
            if extra_pos == -1:
                username = url.split('/')[-1].strip()
            else:
                urlb = url[:extra_pos]
                username = urlb.split('/')[-1].strip()
            return username
        else:
            return None
    else:
        return None

La función recibe como entrada los datos de una persona, el primer paso es checar que el campo 'social_network_accounts' exista. Si no existe nuestra función regresa 'None' inmediatamente. 

En aquellos casos en donde exista el campo entonces suponemos que contendrá una lista con la información de cada una de las cuentas de redes sociales de la persona (tendríamos que checar esto realmente, pero no lo haremos). 

Sobre esta lista podemos ejecutar un filtro usando una función lambda que devuelva solamente aquellos elementos de la lista para los cuales el campo 'value' contiene la cadena 'twitter.com'. 

Si la longitud de la lista filtrada es mayor o igual a uno, es decir, si se encontró al menos un elemento que pertenezca a una cuenta de twitter entonces obtenemos la url de ese elemento, la url se encuentra guardada en el campo 'value'. 

La estrategía a seguir para conseguir el nombre será partir esta URL en pedazos separados por el caracter '/' y tomar el último como el correspondiente al nombre de usuario. Sin embargo antes debemos considerar el caso de que la URL podría tener parámetros asociados que estorben en nuestra estrategía, por ejemplo me encontré algunos con info del idioma de la cuenta. Para quitar estos parámetros antes buscaremos en la URL el caracter '?' y si lo encontramos entonces usaremos su posición para filtrar la URL tomando únicamente la parte de la cadena que se encuentra antes del caracter '?'.

Por último regresamos el pedazo de cadena correspondiente al nombre de usuario.

Probando nuestra función con datos de una persona que si tiene asociada una cuenta de twitter obtenemos su nombre de usuario

In [81]:
usuario_twitter(datos['persons'][4])

'Jorge_HankRhon'

Probando nuestra función con datos de una persona que no tiene asociada una cuenta de twitter ontenemos None

In [82]:
usuario_twitter(datos['persons'][1])

Ahora podemos iterar sobre todas las personas de la API electoral y obtener los nombres de usuario en los casos en los cuales existan. para cada una de las personas con un nombre de usuario válido guardaremos el id de la persona (id de la API electoral para después poder emparentar esta info con los datos de la API) y el nombre de usuario de twitter.

In [83]:
personas_en_twitter = []
for persona in datos['persons']:
    username = usuario_twitter(persona)
    if username is not None:
        personas_en_twitter.append({'id':persona['id'], 'twitter_username':username})

Después de ejecutar este ciclo podemos ver que tenemos 1084 personas con cuenta de twitter.

In [85]:
len(personas_en_twitter)

1084

Podríamos hacer una petición a la API de twitter por cada una de estas personas, sin embargo la API de twitter tiene la capacidad de pedir datos de hasta 100 usuarios en la misma petición. Para poder hacer una petición de varios usuarios es necesario agregar los nombres de usuario de todos los usuarios deseados en los parámetros de la petición. hagamos una función que nos permita estar creando el dict de parámetros para diferentes usuarios. Nuestra función recibirá listas de parámetros y construira el dict necesario con los parámetros de la petición.

In [154]:
def crear_parametros(usernames, expansions, tweet_fields, user_fields):
    usernames_param = ','.join(usernames)
    expansions_param = ','.join(expansions)
    tweet_params = ','.join(tweet_fields)
    user_params = ','.join(user_fields)
    return {'usernames':usernames_param,
        "expansions": expansions_param,
        "tweet.fields": tweet_params,
        "user.fields": user_params}

Algunas de estas listas ya las habíamos construido previamente, probemos nuestra función:

In [155]:
crear_parametros(usernames,expansions,tweet_fields,user_fields)

{'usernames': 'LupJonesof',
 'expansions': 'pinned_tweet_id',
 'tweet.fields': 'attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,text',
 'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified'}

Ahora necesitamos tomar todos los nombres de usuario y agruparlos en conjuntos de máximo 100 usuarios para entonces poder generar las listas de usuario y realizar las peticiones. Usaremos un ciclo sobre un rango que va desde 0 hasta el número de usuarios con username en intervalos de 100 para poder tomar trozos de la lista de personas con nombre de usuario y sobre estos trozos ejecutaremos un map para obtener los nombres de usuario en una lista. Estas listas a su vez las guardaremos en otra lista donde acumularemos los grupos de nombres de usuario

In [156]:
grupos_de_usuarios = []
for i in range(0,len(personas_en_twitter),100):
    user_names = list(map(lambda x: x['twitter_username'],personas_en_twitter[i:i+100]))
    grupos_de_usuarios.append(user_names)

Si exploramos el primer elemento de esta lista de listas podemos observar que guarda 100 nombres de usuario

In [157]:
grupos_de_usuarios[0]

['MarinadelPilar',
 'LupJonesof',
 'atilanocarlosmx',
 'Jorge_HankRhon',
 'VictorCastroCos',
 'PanchoPelayoC',
 'jArmidaCastro',
 'DraAndreaGeiger',
 'GAL280306',
 'AleLageSuarez',
 'adonai_carreon',
 'ManuelGoberBCS',
 'chriscastrob',
 'LaydaSansores',
 'EliseoFdzM',
 'brendariosverde',
 'MaruCampos_G',
 'alejandrodiazmd',
 'GOrtizGlez',
 'JloeraJuan',
 'AlfredoLozoyaMX',
 'Mely_Romero',
 'AuroraCruz09',
 'virgili0Mendoza',
 'leonciomoranL8',
 'indira_vizcaino',
 'ClaudiaYanezCen',
 'irmaliliagarzon',
 'Mario_M_Arcos',
 'Zavaleta_Ruth',
 'ManuelNegreteSI',
 'CarlosHerreraSi',
 'Magana_DeLaMora',
 'MercedesCG_',
 'HipolitoMoraMX',
 'CristobalAriaSo',
 'dipglorianunez',
 'nataliarojasin',
 'nayarmayorquinc',
 'FerLarrazabalNL',
 'AdrianDeLaGarza',
 'claraluzflores',
 'samuel_garcias',
 'carolinagarzaNL',
 'daney_siller',
 'JaquesRivera',
 'makugo',
 'AbiArredondo',
 'RaquelRuizdeSa1',
 'KatiaReja',
 'BetyLeon_',
 'CeliaMayaGar',
 'miguelnava_',
 'jcmoficial1',
 'octaviopedroza',
 'RGC_M

Realicemos una petición a la API de twitter con este primer elemento, 100 usuarios, para entender que es lo que recibimos:

In [158]:
parametros = crear_parametros(grupos_de_usuarios[0],expansions,tweet_fields,user_fields)
rtw = requests.request("GET", user_url, headers=twitter_header, params=parametros)

In [159]:
print(rtw.status_code)

200


De igual manera que en la petición anterior nuestra respuesta tiene 2 campos, 'data' e 'includes' y además ahora se agrega un tercer campo llamado 'errors' en donde se reportan los errores que encontró la petición

In [160]:
rtw.json().keys()

dict_keys(['data', 'includes', 'errors'])

Primero exploremos 'data'. Encontraremos que es una lista, que además tiene 98 elementos. Cada uno de estos elementos contiene información de uno de los 100 nombres de usuario que pasamos a nuestra petición.

In [161]:
type(rtw.json()['data'])

list

In [162]:
len(rtw.json()['data'])

98

In [163]:
rtw.json()['data'][1]

{'username': 'LupJonesof',
 'protected': False,
 'description': 'Orgullosa cachanilla, candidata  a la gubernatura de Baja California por la coalición “Va por Baja California”.',
 'created_at': '2010-02-05T01:27:17.000Z',
 'name': 'Lupita Jones',
 'url': '',
 'profile_image_url': 'https://pbs.twimg.com/profile_images/1378604052532101120/nKkwP19z_normal.jpg',
 'verified': True,
 'pinned_tweet_id': '1384017997635809281',
 'id': '111468543',
 'public_metrics': {'followers_count': 160978,
  'following_count': 292,
  'tweet_count': 21818,
  'listed_count': 338}}

Exploremos ahora el campo 'includes'. Encontraremos que es un dict, que tiene un solo campo llamado 'tweets' el cual almacena una lista, esta lista tiene 48 elementos y cada uno de estos elementos es la información de un tweet que está pinneado en alguna de las cuentas de los 100 usuarios que pasamos a nuestra petición.

In [164]:
type(rtw.json()['includes'])

dict

In [165]:
rtw.json()['includes'].keys()

dict_keys(['tweets'])

In [166]:
type(rtw.json()['includes']['tweets'])

list

In [167]:
len(rtw.json()['includes']['tweets'])

48

In [168]:
rtw.json()['includes']['tweets'][0]

{'id': '1384017997635809281',
 'entities': {'hashtags': [{'start': 240, 'end': 253, 'tag': 'EstadoModelo'},
   {'start': 255, 'end': 264, 'tag': 'BCVota21'},
   {'start': 265, 'end': 279, 'tag': 'DebatesBC2021'}],
  'annotations': [{'start': 165,
    'end': 172,
    'probability': 0.9764,
    'type': 'Place',
    'normalized_text': 'Mexicali'}],
  'urls': [{'start': 280,
    'end': 303,
    'url': 'https://t.co/RcRYs8sXqx',
    'expanded_url': 'https://twitter.com/LupJonesof/status/1384017997635809281/video/1',
    'display_url': 'pic.twitter.com/RcRYs8sXqx'}]},
 'attachments': {'media_keys': ['7_1384017876823068687']},
 'public_metrics': {'retweet_count': 112,
  'reply_count': 166,
  'like_count': 273,
  'quote_count': 10},
 'conversation_id': '1384017997635809281',
 'created_at': '2021-04-19T05:36:19.000Z',
 'reply_settings': 'everyone',
 'text': 'El próximo 6 de junio tienes 3 opciones: votar por un candidato que no sabemos si tiene buenas o malas intenciones, elegir a una candidata

Por último exploremos el campo 'errors'. Podemos ver que contiene una lista con 6 elementos. Cada uno de estos elementos es un error que la API de twitter reporta al tratar de satisfacer nuestra petición, en específico podemos ver dos tipos de errores:
- Could not find user with usernames
- Could not find tweet with pinned_tweet_id

Ambos corresponden a errores en los cuales la API no pudo encontrar el recurso que solicitamos.

In [169]:
type(rtw.json()['errors'])

list

In [170]:
len(rtw.json()['errors'])

6

In [171]:
rtw.json()['errors']

[{'value': 'jArmidaCastro',
  'detail': 'Could not find user with usernames: [jArmidaCastro].',
  'title': 'Not Found Error',
  'resource_type': 'user',
  'parameter': 'usernames',
  'resource_id': 'jArmidaCastro',
  'type': 'https://api.twitter.com/2/problems/resource-not-found'},
 {'value': '1391873804792745988',
  'detail': 'Could not find tweet with pinned_tweet_id: [1391873804792745988].',
  'title': 'Not Found Error',
  'resource_type': 'tweet',
  'parameter': 'pinned_tweet_id',
  'resource_id': '1391873804792745988',
  'type': 'https://api.twitter.com/2/problems/resource-not-found'},
 {'value': '1389990163153973251',
  'detail': 'Could not find tweet with pinned_tweet_id: [1389990163153973251].',
  'title': 'Not Found Error',
  'resource_type': 'tweet',
  'parameter': 'pinned_tweet_id',
  'resource_id': '1389990163153973251',
  'type': 'https://api.twitter.com/2/problems/resource-not-found'},
 {'value': '1375236756220612608',
  'detail': 'Could not find tweet with pinned_tweet_i

Ahora necesitamos amarrar la información de los usuarios de twitter obtenida con la información de los tweets pinneados y con la información del id de la persona en la API electoral. Para poder asociar a un usuario un tweet pineado necesitamos ver si en la info del usuario existe el campo 'pinned_tweet_id' y si existe entonces ir a buscar ese registro en los tweets recuperados. Para asociar a un usuario de twitter con su id en la API electoral necesitamos usar el nombre de usuario y buscar en 'personas_en_twitter' un elemento con ese username. Haremos una función que dada la info de un usuario de twitter, la lista de tweets y la lista de personas en la API electoral con cuenta de twitter amarre lo que tenga que amarrar y regrese un dict con toda la info recolectada. La estrategía es similar a lo usado anteriormente, filtraremos las listas a aquellos elementos que satisfagan la condición impuesta.

In [172]:
def agrupar_informacion(usuario,tweets,personas_en_twitter):
    info = {'info_usuario':usuario}
    pinned_tweet = usuario.get('pinned_tweet_id')
    username = usuario.get('username')
    if pinned_tweet is not None:
        tweet = list(filter(lambda x: x['id'] == pinned_tweet, tweets))
        if len(tweet) != 0:
            info['pinned_tweet_info'] = tweet[0]
    if username is not None:
        persona = list(filter(lambda x: x['twitter_username'] == username, personas_en_twitter))
        if len(persona) != 0:
            info['persona_id'] = persona[0]['id']
    return info

Probemos nuestra función, chequemos que regrese un dict y que este tenga las llaves que nos interesan con la info correcta

In [173]:
usuarios = rtw.json()['data']
tweets = rtw.json()['includes']['tweets']

salida = agrupar_informacion(usuarios[1],tweets,personas_en_twitter)
salida.keys()

dict_keys(['info_usuario', 'pinned_tweet_info', 'persona_id'])

In [174]:
salida['info_usuario']

{'username': 'LupJonesof',
 'protected': False,
 'description': 'Orgullosa cachanilla, candidata  a la gubernatura de Baja California por la coalición “Va por Baja California”.',
 'created_at': '2010-02-05T01:27:17.000Z',
 'name': 'Lupita Jones',
 'url': '',
 'profile_image_url': 'https://pbs.twimg.com/profile_images/1378604052532101120/nKkwP19z_normal.jpg',
 'verified': True,
 'pinned_tweet_id': '1384017997635809281',
 'id': '111468543',
 'public_metrics': {'followers_count': 160978,
  'following_count': 292,
  'tweet_count': 21818,
  'listed_count': 338}}

In [175]:
salida['pinned_tweet_info']

{'id': '1384017997635809281',
 'entities': {'hashtags': [{'start': 240, 'end': 253, 'tag': 'EstadoModelo'},
   {'start': 255, 'end': 264, 'tag': 'BCVota21'},
   {'start': 265, 'end': 279, 'tag': 'DebatesBC2021'}],
  'annotations': [{'start': 165,
    'end': 172,
    'probability': 0.9764,
    'type': 'Place',
    'normalized_text': 'Mexicali'}],
  'urls': [{'start': 280,
    'end': 303,
    'url': 'https://t.co/RcRYs8sXqx',
    'expanded_url': 'https://twitter.com/LupJonesof/status/1384017997635809281/video/1',
    'display_url': 'pic.twitter.com/RcRYs8sXqx'}]},
 'attachments': {'media_keys': ['7_1384017876823068687']},
 'public_metrics': {'retweet_count': 112,
  'reply_count': 166,
  'like_count': 273,
  'quote_count': 10},
 'conversation_id': '1384017997635809281',
 'created_at': '2021-04-19T05:36:19.000Z',
 'reply_settings': 'everyone',
 'text': 'El próximo 6 de junio tienes 3 opciones: votar por un candidato que no sabemos si tiene buenas o malas intenciones, elegir a una candidata

In [176]:
salida['persona_id']

3

Ahora podemos iterar esta función sobre todos los usuarios recolectados en nuestra petición y así estructurar la info que deseamos para cada usuario. Esta info la guardaremos en una lista

In [177]:
usuarios = rtw.json()['data']
tweets = rtw.json()['includes']['tweets']

info_usuarios = []
for usuario in usuarios:
    info = agrupar_informacion(usuario,tweets,personas_en_twitter)
    info_usuarios.append(info)

Esta lista contiene la información recabada de las personas que se pudieron encontrar en twitter. 98 usuarios, 98 elementos.

In [178]:
len(info_usuarios)

98

In [179]:
info_usuarios[0]

{'info_usuario': {'username': 'MarinadelPilar',
  'protected': False,
  'description': 'Mamá, esposa, bajacaliforniana y candidata a Gobernadora de mi Estado ¡Sigamos Haciendo Historia!',
  'location': 'Mexicali, Baja California',
  'created_at': '2009-09-26T04:21:36.000Z',
  'name': 'Marina del Pilar',
  'url': '',
  'profile_image_url': 'https://pbs.twimg.com/profile_images/1392903768329449472/rajxmnFI_normal.jpg',
  'verified': True,
  'id': '77395093',
  'public_metrics': {'followers_count': 5675,
   'following_count': 467,
   'tweet_count': 952,
   'listed_count': 62}},
 'persona_id': 1}

Por último podemos iterar la petición de info y el procesado de información en todos los grupos de usuarios que construimos previamente:

In [181]:
info_usuarios = []

for grupo_indice,grupo in enumerate(grupos_de_usuarios):
    parametros = crear_parametros(grupo,expansions,tweet_fields,user_fields)
    rtw = requests.request("GET", user_url, headers=twitter_header, params=parametros)
    print(grupo_indice,rtw.status_code)
    if rtw.status_code == 200:
        usuarios = rtw.json()['data']
        tweets = rtw.json()['includes']['tweets']

        for usuario in usuarios:
            info = agrupar_informacion(usuario,tweets,personas_en_twitter)
            info_usuarios.append(info)

0 200
1 200
2 200
3 200
4 200
5 200
6 400
7 200
8 200
9 200
10 200


Como podemos observar todas nuestras peticiones salieron bien excepto la petición para el grupo 6, volvamos a realizar esta petición y veamos que sucedió:

In [182]:
parametros = crear_parametros(grupos_de_usuarios[6],expansions,tweet_fields,user_fields)
rtw = requests.request("GET", user_url, headers=twitter_header, params=parametros)

In [183]:
rtw.status_code

400

Exploremos que viene en la petición. El dict regresado tiene 4 campos, 'errors', 'title', 'detail' y 'type'. Veamoslos uno por uno:

In [188]:
rtw.json().keys()

dict_keys(['errors', 'title', 'detail', 'type'])

In [189]:
rtw.json()['errors']

[{'parameters': {'usernames': ['soymauprieto,octavioocampo21,godinez_enrique,berejuarezn,adriana_camposh,laloorihuela_,EdnaDiazMx,CQuintanaMtz,MacarenaChavezM,PacoHuacus,LaraNadiaLuz,ArquiAlbarranc,VeronicaMtzV05,AlejandraPani1,_BrendaEspinoza,luciosantanaz,JulietaMejia,JasminBugarin,camiloramirez,Poblette_Mejia,ThelmaCoraGarza,hugo_leivar,dralilygarciaNL,JesusMarioGarza,mirbeht,RudyMartinezSoy,JudithGraceNL,licmiltongarza,robertosalinasp,abuelocruz14,JOSEHECTORSAN13,DavidToache,SoffyWood,ismaelguerrapa4,ClauHerediaMx,afierro749,guillealvaradom,SaaantiagoGzzs,Yarely_vera1,paolagzzcas,AndresCantuRmz,MarcelaGuerraNLL,raul_alcala,esperanzaliciar,arturo_carmona,joseluisgarza8a,JacobReynaGal,rociomontalvoad,dantepaeznl,GloriaMuiz14,alberto11551575,karlitacarrasco,Yola_MartinezL,angelandariego1,CarolAntonioA,HitaOrtiz1,yas_ramirez_lop,RaquelZapoteca,CONCEPC25666044,irmajuancarlos,DanielGtz_Oax,CesarTL,arleneriveraes1,antonioamaroc,SofiaCastroRios,liborioH_Miriam,Chepi_Org,pepeestefang,Benjam

In [191]:
rtw.json()['title']

'Invalid Request'

In [192]:
rtw.json()['detail']

'One or more parameters to your request was invalid.'

In [193]:
rtw.json()['type']

'https://api.twitter.com/2/problems/invalid-request'

En el campo 'message' de cada elemento en el campo 'errors' podemos ver cual es el problema

In [199]:
rtw.json()['errors'][0]['message']

'The `usernames` query parameter value [MarcelaGuerraNLL] does not match ^[A-Za-z0-9_]{1,15}$'

El nombre de usuario 'MarcelaGuerraNLL' no satisface el formato de peticiones de la API. Una manera simple de resolverlo es eliminandolo del grupo de usuarios

In [201]:
grupos_de_usuarios[6].remove('MarcelaGuerraNLL')

Después de removerlo podemos repetir la petición

In [203]:
parametros = crear_parametros(grupos_de_usuarios[6],expansions,tweet_fields,user_fields)
rtw = requests.request("GET", user_url, headers=twitter_header, params=parametros)

In [204]:
rtw.status_code

200

La petición ha sido respondida satisfactoriamente, ahora podemos procesar los datos obtenidos y agregarlos a nuestra lista de datos procesados 'info_usuarios'

In [205]:
usuarios = rtw.json()['data']
tweets = rtw.json()['includes']['tweets']

for usuario in usuarios:
    info = agrupar_informacion(usuario,tweets,personas_en_twitter)
    info_usuarios.append(info)

Por último podemos explorar cuantos usuarios tenemos en nuestra lista 'info_usuarios'

In [206]:
len(info_usuarios)

1067

Y exportarlo a un archivo de salida:

In [207]:
with open('./datos_usuarios_twitter.json', 'w') as outfile:
    json.dump(info_usuarios, outfile, indent=4, sort_keys=True)